In [1]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

### Proof of the reward structure for each round

[16, 69, 0.5, 32, 0]

[12, 71, 0.5, 32, 1]

[16, 59, 0.5, 32, 2]

[14, 29, 0.75, 32, 3]

[16, 33, 0.75, 32, 4]

[14, 27, 0.75, 32, 5]

[14, 72, 0.5, 40, 6]

[16, 71, 0.5, 40, 7]

[20, 72, 0.5, 40, 8]

[12, 34, 0.75, 40, 9]

[14, 47, 0.75, 40, 10]

[12, 23, 0.75, 40, 11]

[14, 72, 0.5, 48, 12]

[14, 68, 0.5, 48, 13]

[18, 77, 0.5, 48, 14]

[16, 35, 0.75, 48, 15]

[12, 29, 0.75, 48, 16]

[16, 32, 0.75, 48, 17]

Completed all simulations, results saved.

In [2]:
df = pd.read_csv('prisoners_dilemma_results_all_experiments_the_correct_one.csv')

In [16]:
number_of_games = {(0.5, 32) : [69, 71, 59], (0.5, 40) : [72, 71, 72], (0.5, 48) : [72, 68, 77],
                   (0.75, 32) : [29, 33, 27], (0.75, 40) : [34, 47, 23], (0.75, 48) : [35, 29, 32]}

In [19]:
sum([sum(i) for i in number_of_games.values()])

920

In [3]:
df

,Data,Experiment,Supergame,Subject Pair
0,"{'Round': 1, 'Prisoner 1 ID': 11, 'Prisoner 2 ...",0,1,1
1,"{'Round': 2, 'Prisoner 1 ID': 11, 'Prisoner 2 ...",0,1,1
2,"{'Round': 3, 'Prisoner 1 ID': 11, 'Prisoner 2 ...",0,1,1
3,"{'Round': 1, 'Prisoner 1 ID': 0, 'Prisoner 2 I...",0,1,2
4,"{'Round': 2, 'Prisoner 1 ID': 0, 'Prisoner 2 I...",0,1,2
...,...,...,...,...
2512,"{'Round': 3, 'Prisoner 1 ID': 15, 'Prisoner 2 ...",17,4,7
2513,"{'Round': 4, 'Prisoner 1 ID': 15, 'Prisoner 2 ...",17,4,7
2514,"{'Round': 5, 'Prisoner 1 ID': 15, 'Prisoner 2 ...",17,4,7
2515,"{'Round': 1, 'Prisoner 1 ID': 4, 'Prisoner 2 I...",17,4,8


In [4]:
df['Data'][0]

"{'Round': 1, 'Prisoner 1 ID': 11, 'Prisoner 2 ID': 8, 'Prisoner 1 Decision': 'Betray.', 'Prisoner 2 Decision': 'Cooperate.', 'Prisoner 1 Score': 50.0, 'Prisoner 2 Score': 12.0, 'Prisoner 1 LLM Response': 'Betray', 'Prisoner 2 LLM Response': 'Cooperate', 'Probability of Game Continuing': 0.5}"

In [5]:
subjects = {(0.5, 32) : [16, 12, 16], (0.5, 40) : [14, 16, 20], (0.5, 48) : [14, 14, 18],
            (0.75, 32) : [14, 16, 14], (0.75, 40) : [12, 14, 12], (0.75, 48) : [16, 12, 16]}

In [6]:
# Function to get reward based on experiment and supergame
def get_reward(experiment):
    if experiment <= 5:
        return 32
    elif experiment >= 6 and experiment <= 11:
        return 40
    elif experiment >= 12 and experiment <= 17:
        return 48
    return None  # If no match found, return None

In [7]:
df['Reward'] = df.apply(lambda row: get_reward(row['Experiment']), axis=1)

In [8]:
import ast

In [9]:
pd.array([ast.literal_eval(i)['Prisoner 1 LLM Response'] for i in df['Data']]).unique()

<StringArray>
['Betray', 'Cooperate']
Length: 2, dtype: string

In [10]:
# Convert the 'Data' column from string to dictionary using ast.literal_eval
df['Data'] = df['Data'].apply(ast.literal_eval)

# Extract the relevant columns (decisions, scores, etc.) into separate columns
df['Round'] = df['Data'].apply(lambda x: x['Round'])
df['Prisoner 1 Decision'] = df['Data'].apply(lambda x: x['Prisoner 1 LLM Response'])
df['Prisoner 2 Decision'] = df['Data'].apply(lambda x: x['Prisoner 2 LLM Response'])
df['Prisoner 1 Score'] = df['Data'].apply(lambda x: x['Prisoner 1 Score'])
df['Prisoner 2 Score'] = df['Data'].apply(lambda x: x['Prisoner 2 Score'])

In [11]:
df['Probability of Game Continuing'] = df['Data'].apply(lambda x: x['Probability of Game Continuing'])

In [50]:
df['Data'][0]['Prisoner 1 LLM Response']

'Betray'

In [55]:
type(df['Prisoner 1 Score'][0])

numpy.float64

In [15]:
sum(df['Round'] == 1)

964

In [56]:
# Now let's define a function to classify decisions: 1 for Cooperation, 0 for Defection
def classify_decision(decision):
    if decision == 'Cooperate':
        return 1
    elif decision == 'Betray':
        return 0
    else:
        return None  # Handle other decision values if needed

# Apply the classification to the 'Prisoner 1 Decision' and 'Prisoner 2 Decision'
df['Player 1 Cooperates'] = df['Prisoner 1 Decision'].apply(classify_decision)
df['Player 2 Cooperates'] = df['Prisoner 2 Decision'].apply(classify_decision)

# Calculate cooperation rates per round
cooperation_rate_per_round = df.groupby('Round')[['Player 1 Cooperates', 'Player 2 Cooperates']].mean()

# Print the cooperation rates per round
print(cooperation_rate_per_round)

       Player 1 Cooperates  Player 2 Cooperates
Round                                          
1                 0.578838             0.539419
2                 0.614565             0.606762
3                 0.622356             0.610272
4                 0.622549             0.534314
5                 0.613636             0.621212
6                 0.606061             0.696970
7                 0.550725             0.521739
8                 0.612245             0.612245
9                 0.729730             0.540541
10                0.523810             0.666667
11                0.600000             0.600000
12                0.454545             0.363636
13                0.500000             0.750000
14                0.400000             0.800000
15                0.250000             0.250000
16                0.333333             1.000000
17                1.000000             0.500000


In [62]:
cooperation_rate

,Player 1 Cooperates,Player 2 Cooperates
Probability of Game Continuing,,
0.50,0.608470,0.599554
0.75,0.588386,0.550427


In [63]:
# Define function to compute cooperation rate
def cooperation_rate(group):
    return group[['Player 1 Cooperates', 'Player 2 Cooperates']].mean().mean()

# Group by Reward and Probability of Game Continuing and compute average cooperation rate
cooperation_rate_by_reward_and_probability = df.groupby(['Reward', 'Probability of Game Continuing']).apply(cooperation_rate).reset_index(name='Cooperation Rate')

# Set threshold for cooperation classification
cooperation_rate_by_reward_and_probability['Cooperation Status'] = cooperation_rate_by_reward_and_probability['Cooperation Rate'].apply(lambda x: 'YES' if x > 0.5 else 'NO')

# Pivot the table to get a clear representation of cooperation in equilibrium
table = cooperation_rate_by_reward_and_probability.pivot_table(index='Reward', columns='Probability of Game Continuing', values='Cooperation Status', aggfunc='first')

# Print the table
print(table)

Probability of Game Continuing 0.50 0.75
Reward                                  
32                              YES  YES
40                              YES  YES
48                              YES  YES


C:\Users\ondre\AppData\Local\Temp\ipykernel_3180\2424640953.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cooperation_rate_by_reward_and_probability = df.groupby(['Reward', 'Probability of Game Continuing']).apply(cooperation_rate).reset_index(name='Cooperation Rate')


In [64]:
df

,Data,Experiment,Supergame,Subject Pair,Reward,Round,Prisoner 1 Decision,Prisoner 2 Decision,Prisoner 1 Score,Prisoner 2 Score,Player 1 Cooperates,Player 2 Cooperates,Probability of Game Continuing
0,"{'Round': 1, 'Prisoner 1 ID': 11, 'Prisoner 2 ...",0,1,1,32,1,Betray,Cooperate,50.0,12.0,0,1.0,0.50
1,"{'Round': 2, 'Prisoner 1 ID': 11, 'Prisoner 2 ...",0,1,1,32,2,Betray,Cooperate,100.0,24.0,0,1.0,0.50
2,"{'Round': 3, 'Prisoner 1 ID': 11, 'Prisoner 2 ...",0,1,1,32,3,Cooperate,Cooperate,132.0,56.0,1,1.0,0.50
3,"{'Round': 1, 'Prisoner 1 ID': 0, 'Prisoner 2 I...",0,1,2,32,1,Betray,Betray,25.0,25.0,0,0.0,0.50
4,"{'Round': 2, 'Prisoner 1 ID': 0, 'Prisoner 2 I...",0,1,2,32,2,Betray,Betray,50.0,50.0,0,0.0,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2512,"{'Round': 3, 'Prisoner 1 ID': 15, 'Prisoner 2 ...",17,4,7,48,3,Betray,Betray,378.0,314.0,0,0.0,0.75
2513,"{'Round': 4, 'Prisoner 1 ID': 15, 'Prisoner 2 ...",17,4,7,48,4,Betray,Cooperate,428.0,326.0,0,1.0,0.75
2514,"{'Round': 5, 'Prisoner 1 ID': 15, 'Prisoner 2 ...",17,4,7,48,5,Cooperate,Cooperate,476.0,374.0,1,1.0,0.75
2515,"{'Round': 1, 'Prisoner 1 ID': 4, 'Prisoner 2 I...",17,4,8,48,1,Betray,Cooperate,796.0,417.0,0,1.0,0.75
